Now to try to use our own GAT Pytorch implementation. Let's see how it goes.

In [101]:
import numpy as np
import torch_gat
import torch_geometric
import torch
import importlib
importlib.reload(torch_gat)

<module 'torch_gat' from '/home/ruy/Downloads/CS598-final/code/torch_gat.py'>

In [ ]:
for dataset in ['citeseer', 'cora']:
    data = torch_geometric.datasets.Planetoid(root=f'../data/{dataset}', name=dataset)
    bias = torch_gat.adj_to_bias(torch_gat.planetoid_adj_to_petarv_adj(data.edge_index).todense()[np.newaxis], [data.x.shape[0]], nhood=1)
    model = torch_gat.GAT(data.num_features, data.num_classes, bias)
    optimizer = torch.optim.Adam(model.parameters(), lr=.005, weight_decay=5e-4)

    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        out = model(data.x[np.newaxis]).squeeze()
        loss = torch.nn.functional.cross_entropy(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(out.argmax(dim=1)[data.train_mask].eq(data.y[data.train_mask]).sum().item() / int(data.train_mask.sum()))

    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    acc = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item() / int(data.test_mask.sum())
    print('\n\n*****************************************************************************************************\n')
    print(f'                                         {dataset} ')
    print(f'                                         Total Epochs: 200')
    print(f'                                         Test Accuracy: {acc:.4f}')
    print('\n*****************************************************************************************************\n\n')

0.2
0.2833333333333333
0.24166666666666667
0.18333333333333332
0.3
0.23333333333333334
0.19166666666666668
0.18333333333333332
0.25
0.2
0.25833333333333336
0.20833333333333334
0.175
0.2
0.175
0.225
0.175
